# Supplementary material for Q11

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.0.1


## 1) Convolutional neural network architecture

In [68]:
class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 5, kernel_size=5, stride=1), # 5 * (5*5 * 3) + 5
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=2),
            torch.nn.Conv2d(5, 12, kernel_size=3, stride=1),  # 12 * (3*3 * 5) + 12
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=3, stride=2),
            torch.nn.ReLU(),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(192, 128), # 192 * 128 + 128
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes), # 128 * 10 + 10
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

## 2) Computing the number of parameters

### 2 a) By hand

In [69]:
# convolutional part
conv_part =  (5 * (5*5 * 3) + 5 ) + ( 12 * (3*3 * 5) + 12 )
print(conv_part)

932


In [70]:
# fully connected part
fc_part = 192*128+128 + 128*10+10
print(fc_part)

25994


In [71]:
# total
print(conv_part + fc_part)

26926


### 2 b) Adding .parameters() programmatically

In [72]:
model = PyTorchCNN(10)

def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [73]:
# convolutional part
count_parameters(model.features)

932

In [74]:
# fully connected part
count_parameters(model.classifier)

25994

In [75]:
# total
count_parameters(model)

26926

### 2 c) Computer the memory size

In [97]:
import sys

def calculate_size(model): 
    return sum(p.element_size()*p.numel() for p in model.parameters())

size_in_bytes = calculate_size(model)
size_in_megabytes = size_in_bytes * 1e-6

print(f"{size_in_megabytes: .2f} Mb")

 0.11 Mb


### 2 d) Using the torchinfo library

In [99]:
# using https://github.com/TylerYep/torchinfo
# pip install torchinfo

import torchinfo

print(f"Torchinfo version: {torchinfo.__version__}")

batch_size = 16
torchinfo.summary(model, input_size=(batch_size, 3, 32, 32))

Torchinfo version: 1.7.2


Layer (type:depth-idx)                   Output Shape              Param #
PyTorchCNN                               [16, 10]                  --
├─Sequential: 1-1                        [16, 12, 4, 4]            --
│    └─Conv2d: 2-1                       [16, 5, 28, 28]           380
│    └─ReLU: 2-2                         [16, 5, 28, 28]           --
│    └─MaxPool2d: 2-3                    [16, 5, 12, 12]           --
│    └─Conv2d: 2-4                       [16, 12, 10, 10]          552
│    └─ReLU: 2-5                         [16, 12, 10, 10]          --
│    └─AvgPool2d: 2-6                    [16, 12, 4, 4]            --
│    └─ReLU: 2-7                         [16, 12, 4, 4]            --
├─Sequential: 1-2                        [16, 10]                  --
│    └─Flatten: 2-8                      [16, 192]                 --
│    └─Linear: 2-9                       [16, 128]                 24,704
│    └─ReLU: 2-10                        [16, 128]                 --
│    └─Li

## 3) ADAM optimizer

In [47]:
optimizer = torch.optim.Adam(model.parameters())

In [56]:
optimizer.param_groups.count

<function list.count(value, /)>

In [104]:
sum(p.numel() for p in optimizer.param_groups[0]['params'])

26926

## 4) BatchNorm

In [107]:
class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 5, kernel_size=5, stride=1),
            torch.nn.BatchNorm2d(5),  # NEW!
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=2),
            torch.nn.Conv2d(5, 12, kernel_size=3, stride=1),
            torch.nn.BatchNorm2d(12),  # NEW!
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=3, stride=2),
            torch.nn.ReLU(),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(192, 128),
            torch.nn.BatchNorm1d(128),  # NEW!
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [108]:
model = PyTorchCNN(10)

torchinfo.summary(model, input_size=(batch_size, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
PyTorchCNN                               [16, 10]                  --
├─Sequential: 1-1                        [16, 12, 4, 4]            --
│    └─Conv2d: 2-1                       [16, 5, 28, 28]           380
│    └─BatchNorm2d: 2-2                  [16, 5, 28, 28]           10
│    └─ReLU: 2-3                         [16, 5, 28, 28]           --
│    └─MaxPool2d: 2-4                    [16, 5, 12, 12]           --
│    └─Conv2d: 2-5                       [16, 12, 10, 10]          552
│    └─BatchNorm2d: 2-6                  [16, 12, 10, 10]          24
│    └─ReLU: 2-7                         [16, 12, 10, 10]          --
│    └─AvgPool2d: 2-8                    [16, 12, 4, 4]            --
│    └─ReLU: 2-9                         [16, 12, 4, 4]            --
├─Sequential: 1-2                        [16, 10]                  --
│    └─Flatten: 2-10                     [16, 192]                 --
│    └─Linear